In [1]:
# Core libraries
import pandas as pd
import numpy as np
import re

# ML & text processing
from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMRegressor
from scipy.sparse import hstack


In [2]:
# Load train and test CSV files
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Display first 5 rows to check data
print("Train DataFrame head:")
display(train_df.head())

print("\nTest DataFrame head:")
display(test_df.head())


FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [3]:
import os
os.getcwd()



'C:\\Rahul\\GitHub\\AmazonML-Hackathon'

In [4]:
# Load train and test CSV files
train_df = pd.read_csv(r'C:\Rahul\GitHub\AmazonML-Hackathon\dataset\train.csv')
test_df = pd.read_csv(r'C:\Rahul\GitHub\AmazonML-Hackathon\dataset\test.csv')

# Display first 5 rows to check data
print("Train DataFrame head:")
display(train_df.head())

print("\nTest DataFrame head:")
display(test_df.head())


Train DataFrame head:


,sample_id,catalog_content,image_link,price
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49



Test DataFrame head:


,sample_id,catalog_content,image_link
0,100179,Item Name: Rani 14-Spice Eshamaya's Mango Chut...,https://m.media-amazon.com/images/I/71hoAn78AW...
1,245611,Item Name: Natural MILK TEA Flavoring extract ...,https://m.media-amazon.com/images/I/61ex8NHCIj...
2,146263,Item Name: Honey Filled Hard Candy - Bulk Pack...,https://m.media-amazon.com/images/I/61KCM61J8e...
3,95658,Item Name: Vlasic Snack'mm's Kosher Dill 16 Oz...,https://m.media-amazon.com/images/I/51Ex6uOH7y...
4,36806,"Item Name: McCormick Culinary Vanilla Extract,...",https://m.media-amazon.com/images/I/71QYlrOMoS...


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import re
import numpy as np

# Optional: extract quantity (numbers in catalog) for extra feature
def extract_quantity(text):
    nums = re.findall(r'\d+', str(text))
    return int(nums[-1]) if nums else 1

train_df['quantity'] = train_df['catalog_content'].apply(extract_quantity)
test_df['quantity'] = test_df['catalog_content'].apply(extract_quantity)

# TF-IDF vectorization (tokenization is automatic)
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1,2))
X_train_text = vectorizer.fit_transform(train_df['catalog_content'])
X_test_text = vectorizer.transform(test_df['catalog_content'])

# Combine text features with quantity
X_train = hstack((X_train_text, np.array(train_df['quantity']).reshape(-1,1)))
X_test = hstack((X_test_text, np.array(test_df['quantity']).reshape(-1,1)))
y_train = train_df['price']


In [6]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(n_estimators=800, learning_rate=0.05)
model.fit(X_train, y_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.909318 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 810788
[LightGBM] [Info] Number of data points in the train set: 75000, number of used features: 5001
[LightGBM] [Info] Start training from score 23.647654


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.05
,n_estimators,800
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [7]:
predictions = model.predict(X_test)
predictions = np.maximum(predictions, 0.01)  # ensure no negative prices


C:\Rahul\GitHub\AmazonML-Hackathon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Rahul\GitHub\AmazonML-Hackathon\venv\Lib\site-packages\lightgbm\basic.py:1238: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


In [8]:
import warnings
warnings.filterwarnings("ignore")


In [9]:
submission = pd.DataFrame({
    'sample_id': test_df['sample_id'],
    'price': predictions
})

submission.to_csv('test_out.csv', index=False)
print("Submission file saved: test_out.csv")

Submission file saved: test_out.csv
